In [ ]:
# default_exp funcs.emitter_io

In [ ]:
%load_ext autoreload
%autoreload 2

# Functions for working with emitter DFs

In [ ]:
#export
from decode_fish.imports import *

In [ ]:
#export
def shift_df(df, shift=[0.,0.,0.]):
    
    df_corr = df.copy()
    df_corr['x'] += shift[0]
    df_corr['y'] += shift[1]
    df_corr['z'] += shift[2]
    return df_corr

def sig_filt(df, perc = 90, return_low=True):
    if perc >= 100:
        return df
    filt_val = np.percentile(df['comb_sig'], perc)
    if return_low:
        return df[df['comb_sig'] < filt_val]
    else:
        return df[df['comb_sig'] > filt_val]

#export
def nm_to_px(df, px_size_zyx=[100.,100.,100.]):
    
    if df is None: return None
    df_corr = df.copy()
    df_corr['x'] /= px_size_zyx[2]
    df_corr['y'] /= px_size_zyx[1]
    df_corr['z'] /= px_size_zyx[0]
    return df_corr

#export
def px_to_nm(df, px_size_zyx=[100.,100.,100.]):
    
    df_corr = df.copy()
    df_corr['x'] *= px_size_zyx[2]
    df_corr['y'] *= px_size_zyx[1]
    df_corr['z'] *= px_size_zyx[0]
    return df_corr

#export
def cat_emitter_dfs(df_list, n_frames=1):
    
    if isinstance(n_frames, int):
        n_frames = [n_frames for _ in df_list]
    cum_frames = np.cumsum(n_frames)
    ret_df = df_list[0]
    for df, n_f in zip(df_list[1:], cum_frames[:-1]):
        dfc = df.copy()
        if len(ret_df):
            dfc['frame_idx'] += n_f
            dfc['loc_idx'] += ret_df['loc_idx'].values[-1] + 1
        ret_df = pd.concat([ret_df, dfc], ignore_index=True)
    return ret_df

#export
def crop_df(df, fzyx_sl=np.s_[:,:,:,:], shift=True, px_size_zyx=[1.,1.,1.]):
    
    df_crop = df.copy()
    for sl, key, px_s in zip(fzyx_sl, ['frame_idx','z','y','x'], [1] + px_size_zyx):
        if sl.start:
            df_crop = df_crop[df_crop[key] > px_s*sl.start]
        if sl.stop:
            df_crop = df_crop[df_crop[key] < px_s*sl.stop]
        if (shift) & (sl.start is not None):
            df_crop[key] -= px_s*sl.start
        
    return df_crop

def get_n_locs(df):
    counts = []
    vcounts = pred_df['frame_idx'].value_counts()
    for i in range(pred_df['frame_idx'].max()+1):
        if i in vcounts.index:
            counts.append(vcounts[i])
    return counts

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted index.ipynb.
